# 🚀 Proyecto ShuffleNet - Transfer Learning
## INFO1185 - Inteligencia Artificial III
### Clasificación de Vegetales con ShuffleNet V2

**Autor:** Benja  
**Año:** 2025

---

## 📋 Descripción del Proyecto

Este proyecto implementa Transfer Learning usando **ShuffleNet V2** preentrenado en ImageNet para clasificar **5 tipos de vegetales**:

1. 🌶️ Jalapeño
2. 🌶️ Chili Pepper
3. 🥕 Carrot
4. 🌽 Corn
5. 🥒 Cucumber

### Características Principales:
- ✅ Modelo base: ShuffleNet V2 x1.0 (preentrenado en ImageNet)
- ✅ Feature extractor congelado
- ✅ Clasificador simple (1 capa FC)
- ✅ Dataset dividido en train/val/test
- ✅ Data augmentation en entrenamiento

## 📦 Paso 1: Instalación de Dependencias

**Nota:** Si estás en Google Colab, ejecuta esta celda. Si ya tienes las librerías instaladas, puedes saltarla.

In [1]:
# Instalación de paquetes necesarios
!pip install torch torchvision tqdm matplotlib numpy pillow -q

print("✅ Librerías instaladas correctamente!")

✅ Librerías instaladas correctamente!



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 📚 Paso 2: Importar Librerías

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

print("✅ PyTorch version:", torch.__version__)
print("✅ Torchvision version:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

✅ PyTorch version: 2.9.1+cpu
✅ Torchvision version: 2.9.1+cpu
✅ CUDA available: False


## 🗂️ Paso 3: Clase de Preparación de Datos

Esta clase maneja:
- Carga del dataset
- Filtrado de las 5 clases específicas
- Transformaciones (data augmentation para train, normalización para val/test)
- Creación de DataLoaders

In [13]:
class DataPreparation:
    """
    Clase para preparar y cargar el dataset.
    Filtra 5 clases específicas: jalepeno, chilli pepper, carrot, corn, cucumber.
    """
    
    def __init__(self, data_dir="./archive", batch_size=32):
        """
        Inicializa el preparador de datos.
        
        Args:
            data_dir (str): Directorio raíz del dataset (default: ./archive)
            batch_size (int): Tamaño del batch para los DataLoaders
        """
        self.data_dir = data_dir
        self.batch_size = batch_size
        
        # Las 5 clases que necesitamos
        self.selected_classes = [
            'jalepeno',
            'chilli pepper',
            'carrot',
            'corn',
            'cucumber'
        ]
        
        # Rutas de train, val y test
        self.train_dir = os.path.join(data_dir, 'train')
        self.val_dir = os.path.join(data_dir, 'validation')
        self.test_dir = os.path.join(data_dir, 'test')
    
    def get_train_transforms(self):
        """
        Transformaciones para entrenamiento con data augmentation.
        """
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.1
            ),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    
    def get_val_test_transforms(self):
        """
        Transformaciones para validación y test sin augmentation.
        """
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    
    def create_filtered_dataset(self, dataset):
        """
        Crea un dataset filtrado con labels remapeados de 0 a 4.
        
        Args:
            dataset: Dataset original de PyTorch
            
        Returns:
            tuple: (samples filtrados, mapeo de labels)
        """
        class_to_idx = dataset.class_to_idx
        
        # Obtener índices originales de las clases seleccionadas
        selected_indices = {class_to_idx[cls]: i for i, cls in enumerate(self.selected_classes) if cls in class_to_idx}
        
        # Filtrar samples y remapear labels
        filtered_samples = []
        for path, label in dataset.samples:
            if label in selected_indices:
                new_label = selected_indices[label]  # Remapear a 0-4
                filtered_samples.append((path, new_label))
        
        return filtered_samples
    
    def create_dataloaders(self):
        """
        Crea los DataLoaders para train, validation y test.
        
        Returns:
            tuple: (train_loader, val_loader, test_loader, num_classes, class_names)
        """
        print("="*70)
        print("📦 PREPARANDO DATOS")
        print("="*70)
        
        # Crear datasets completos (sin transformaciones primero para filtrar)
        train_dataset_full = datasets.ImageFolder(root=self.train_dir)
        val_dataset_full = datasets.ImageFolder(root=self.val_dir)
        test_dataset_full = datasets.ImageFolder(root=self.test_dir)
        
        print(f"\n📊 Dataset completo:")
        print(f"   - Total de clases: {len(train_dataset_full.classes)}")
        print(f"   - Train: {len(train_dataset_full)} imágenes")
        print(f"   - Val: {len(val_dataset_full)} imágenes")
        print(f"   - Test: {len(test_dataset_full)} imágenes")
        
        # Filtrar y remapear labels
        print(f"\n🔍 Filtrando solo las 5 clases requeridas...")
        train_samples = self.create_filtered_dataset(train_dataset_full)
        val_samples = self.create_filtered_dataset(val_dataset_full)
        test_samples = self.create_filtered_dataset(test_dataset_full)
        
        print(f"\n✅ Dataset filtrado (5 clases):")
        print(f"   - Clases: {self.selected_classes}")
        print(f"   - Train: {len(train_samples)} imágenes")
        print(f"   - Val: {len(val_samples)} imágenes")
        print(f"   - Test: {len(test_samples)} imágenes")
        
        # Crear datasets con transformaciones y samples filtrados
        train_dataset = datasets.ImageFolder(root=self.train_dir, transform=self.get_train_transforms())
        train_dataset.samples = train_samples
        train_dataset.imgs = train_samples
        train_dataset.targets = [s[1] for s in train_samples]
        
        val_dataset = datasets.ImageFolder(root=self.val_dir, transform=self.get_val_test_transforms())
        val_dataset.samples = val_samples
        val_dataset.imgs = val_samples
        val_dataset.targets = [s[1] for s in val_samples]
        
        test_dataset = datasets.ImageFolder(root=self.test_dir, transform=self.get_val_test_transforms())
        test_dataset.samples = test_samples
        test_dataset.imgs = test_samples
        test_dataset.targets = [s[1] for s in test_samples]
        
        # Crear DataLoaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2,
            pin_memory=True
        )
        
        val_loader = DataLoader(
            val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )
        
        test_loader = DataLoader(
            test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )
        
        print(f"\n📦 DataLoaders creados:")
        print(f"   - Batch size: {self.batch_size}")
        print(f"   - Train batches: {len(train_loader)}")
        print(f"   - Val batches: {len(val_loader)}")
        print(f"   - Test batches: {len(test_loader)}")
        print("="*70)
        
        return train_loader, val_loader, test_loader, 5, self.selected_classes

print("✅ Clase DataPreparation definida!")

✅ Clase DataPreparation definida!


## 🤖 Paso 4: Definición del Modelo ShuffleNet

Clase que implementa:
- Carga de ShuffleNet V2 preentrenado en ImageNet
- Congelamiento del feature extractor
- Clasificador simple (1 capa FC, sin BatchNorm, sin Dropout)

In [5]:
class ShuffleNetSimple(nn.Module):
    """
    ShuffleNet con clasificador simple (Versión 1).
    
    Características:
    - Modelo base: ShuffleNet V2 preentrenado en ImageNet
    - Clasificador: Una sola capa Fully Connected
    - SIN BatchNorm
    - SIN Dropout
    """
    
    def __init__(self, num_classes=5, pretrained=True, freeze_features=True):
        """
        Inicializa el modelo ShuffleNet con clasificador simple.
        
        Args:
            num_classes (int): Número de clases de salida (default: 5)
            pretrained (bool): Si cargar pesos preentrenados de ImageNet (default: True)
            freeze_features (bool): Si congelar las capas convolucionales (default: True)
        """
        super(ShuffleNetSimple, self).__init__()
        
        # Cargar ShuffleNet V2 preentrenado en ImageNet
        print("🔄 Cargando ShuffleNet V2 preentrenado...")
        
        # Usar weights parameter (nuevo API de torchvision >= 0.13)
        try:
            if pretrained:
                self.shufflenet = models.shufflenet_v2_x1_0(
                    weights=models.ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1
                )
            else:
                self.shufflenet = models.shufflenet_v2_x1_0(weights=None)
        except:
            # Fallback para versiones antiguas de torchvision
            self.shufflenet = models.shufflenet_v2_x1_0(pretrained=pretrained)
        
        print("✅ ShuffleNet V2 cargado exitosamente!")
        
        # Congelar o no las capas convolucionales (feature extractor)
        if freeze_features:
            print("❄️  Congelando capas convolucionales (feature extractor)...")
            for param in self.shufflenet.parameters():
                param.requires_grad = False
            print("✅ Capas convolucionales congeladas!")
        else:
            print("🔥 Capas convolucionales entrenable (fine-tuning completo)")
        
        # Obtener el tamaño de entrada del clasificador original
        # En ShuffleNet V2 x1.0, la última capa conv produce 1024 features
        in_features = self.shufflenet.fc.in_features
        
        # 🎯 VERSIÓN 1: CLASIFICADOR SIMPLE
        # Solo una capa Fully Connected
        # SIN BatchNorm
        # SIN Dropout
        self.shufflenet.fc = nn.Linear(in_features, num_classes)
        
        print(f"\n🎯 CLASIFICADOR SIMPLE (Versión 1) creado:")
        print(f"   - Input features: {in_features}")
        print(f"   - Output classes: {num_classes}")
        print(f"   - Capas: 1 Linear")
        print(f"   - BatchNorm: NO")
        print(f"   - Dropout: NO")
    
    def forward(self, x):
        """
        Forward pass del modelo.
        
        Args:
            x (torch.Tensor): Tensor de entrada [batch_size, 3, 224, 224]
        
        Returns:
            torch.Tensor: Logits de salida [batch_size, num_classes]
        """
        return self.shufflenet(x)
    
    def get_trainable_params(self):
        """
        Obtiene los parámetros entrenables del modelo.
        
        Returns:
            list: Lista de parámetros que requieren gradiente
        """
        return [p for p in self.parameters() if p.requires_grad]
    
    def count_parameters(self):
        """
        Cuenta los parámetros del modelo.
        
        Returns:
            dict: Diccionario con total, entrenables y congelados
        """
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        frozen_params = total_params - trainable_params
        
        return {
            'total': total_params,
            'trainable': trainable_params,
            'frozen': frozen_params
        }
    
    def print_model_info(self):
        """
        Imprime información detallada del modelo.
        """
        params = self.count_parameters()
        print("\n" + "="*60)
        print("📊 INFORMACIÓN DEL MODELO")
        print("="*60)
        print(f"Parámetros totales:      {params['total']:,}")
        print(f"Parámetros entrenables:  {params['trainable']:,}")
        print(f"Parámetros congelados:   {params['frozen']:,}")
        print(f"Porcentaje entrenable:   {params['trainable']/params['total']*100:.2f}%")
        print("="*60)

print("✅ Clase ShuffleNetSimple definida!")

✅ Clase ShuffleNetSimple definida!


## ⚙️ Paso 5: Configuración de Parámetros

Definimos todos los hiperparámetros del entrenamiento.

In [14]:
# ==========================================
# CONFIGURACIÓN
# ==========================================

# Ruta del dataset
DATA_DIR = "./archive"  # Cambiar si es necesario

# Parámetros del modelo
NUM_CLASSES = 5  # jalepeno, chilli pepper, carrot, corn, cucumber

# Hiperparámetros de entrenamiento
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

# Dispositivo (GPU si está disponible, sino CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("="*70)
print("📋 CONFIGURACIÓN DEL PROYECTO")
print("="*70)
print(f"✅ Dispositivo: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
print(f"✅ Clases: {NUM_CLASSES}")
print(f"✅ Batch size: {BATCH_SIZE}")
print(f"✅ Learning rate: {LEARNING_RATE}")
print(f"✅ Épocas: {NUM_EPOCHS}")
print("="*70)

📋 CONFIGURACIÓN DEL PROYECTO
✅ Dispositivo: cpu
✅ Clases: 5
✅ Batch size: 32
✅ Learning rate: 0.001
✅ Épocas: 10


## 📥 Paso 6: Cargar Datos

Creamos los DataLoaders para entrenamiento, validación y prueba.

In [15]:
# Crear instancia de DataPreparation
data_prep = DataPreparation(data_dir=DATA_DIR, batch_size=BATCH_SIZE)

# Crear DataLoaders
train_loader, val_loader, test_loader, num_classes, class_names = data_prep.create_dataloaders()

print("\n✅ Datos cargados exitosamente!")
print(f"   Clases: {class_names}")

📦 PREPARANDO DATOS

📊 Dataset completo:
   - Total de clases: 36
   - Train: 3115 imágenes
   - Val: 351 imágenes
   - Test: 359 imágenes

🔍 Filtrando solo las 5 clases requeridas...

✅ Dataset filtrado (5 clases):
   - Clases: ['jalepeno', 'chilli pepper', 'carrot', 'corn', 'cucumber']
   - Train: 438 imágenes
   - Val: 47 imágenes
   - Test: 50 imágenes

📦 DataLoaders creados:
   - Batch size: 32
   - Train batches: 14
   - Val batches: 2
   - Test batches: 2

✅ Datos cargados exitosamente!
   Clases: ['jalepeno', 'chilli pepper', 'carrot', 'corn', 'cucumber']


## 🏗️ Paso 7: Crear Modelo

Inicializamos el modelo ShuffleNet y lo movemos al dispositivo (GPU/CPU).

In [16]:
# Crear modelo
model = ShuffleNetSimple(
    num_classes=NUM_CLASSES,
    pretrained=True,
    freeze_features=True
)

# Mostrar información del modelo
model.print_model_info()

# Mover modelo al dispositivo
model = model.to(device)
print(f"\n✅ Modelo movido a {device}")

🔄 Cargando ShuffleNet V2 preentrenado...
✅ ShuffleNet V2 cargado exitosamente!
❄️  Congelando capas convolucionales (feature extractor)...
✅ Capas convolucionales congeladas!

🎯 CLASIFICADOR SIMPLE (Versión 1) creado:
   - Input features: 1024
   - Output classes: 5
   - Capas: 1 Linear
   - BatchNorm: NO
   - Dropout: NO

📊 INFORMACIÓN DEL MODELO
Parámetros totales:      1,258,729
Parámetros entrenables:  5,125
Parámetros congelados:   1,253,604
Porcentaje entrenable:   0.41%

✅ Modelo movido a cpu


## 🎯 Paso 8: Configurar Entrenamiento

Definimos la función de pérdida, optimizador y scheduler.

In [17]:
# Función de pérdida
criterion = nn.CrossEntropyLoss()
print("✅ Loss function: CrossEntropyLoss")

# Optimizador (solo para parámetros entrenables)
optimizer = optim.Adam(model.get_trainable_params(), lr=LEARNING_RATE)
print(f"✅ Optimizer: Adam (lr={LEARNING_RATE})")

# Scheduler (opcional - reduce LR cada 5 épocas)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
print("✅ Scheduler: StepLR (step=5, gamma=0.1)")

✅ Loss function: CrossEntropyLoss
✅ Optimizer: Adam (lr=0.001)
✅ Scheduler: StepLR (step=5, gamma=0.1)


## 🔄 Paso 9: Funciones de Entrenamiento y Validación

In [18]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """
    Entrena el modelo por una época.
    
    Args:
        model: Modelo a entrenar
        train_loader: DataLoader de entrenamiento
        criterion: Función de pérdida
        optimizer: Optimizador
        device: Dispositivo (cuda/cpu)
    
    Returns:
        tuple: (loss promedio, accuracy)
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Progress bar
    pbar = tqdm(train_loader, desc="Entrenando", leave=False)
    
    for images, labels in pbar:
        # Mover datos al dispositivo
        images, labels = images.to(device), labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Estadísticas
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Actualizar progress bar
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100 * correct / total:.2f}%'
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    """
    Valida el modelo.
    
    Args:
        model: Modelo a validar
        val_loader: DataLoader de validación
        criterion: Función de pérdida
        device: Dispositivo (cuda/cpu)
    
    Returns:
        tuple: (loss promedio, accuracy)
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            # Mover datos al dispositivo
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Estadísticas
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc

print("✅ Funciones de entrenamiento y validación definidas!")

✅ Funciones de entrenamiento y validación definidas!


## 🚀 Paso 10: Loop de Entrenamiento

Entrenamos el modelo durante el número de épocas especificado.

In [19]:
# Listas para guardar métricas
train_losses = []
train_accs = []
val_losses = []
val_accs = []

print("="*70)
print("🚀 INICIANDO ENTRENAMIENTO")
print("="*70)

best_val_acc = 0.0
best_model_state = None

for epoch in range(NUM_EPOCHS):
    print(f"\n📍 Época {epoch+1}/{NUM_EPOCHS}")
    print("-" * 70)
    
    # Entrenamiento
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, device
    )
    
    # Validación
    val_loss, val_acc = validate(
        model, val_loader, criterion, device
    )
    
    # Guardar métricas
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Actualizar learning rate
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    # Imprimir resultados
    print(f"✅ Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"✅ Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"📊 Learning Rate: {current_lr:.6f}")
    
    # Guardar mejor modelo
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        print(f"🌟 ¡Nuevo mejor modelo! Val Acc: {best_val_acc:.2f}%")

print("\n" + "="*70)
print("✅ ENTRENAMIENTO COMPLETADO")
print("="*70)
print(f"🏆 Mejor Val Accuracy: {best_val_acc:.2f}%")

# Cargar el mejor modelo
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("✅ Mejor modelo cargado para evaluación")

🚀 INICIANDO ENTRENAMIENTO

📍 Época 1/10
----------------------------------------------------------------------


✅ Train Loss: 1.5965 | Train Acc: 37.90%
✅ Val Loss:   1.5655 | Val Acc:   63.83%
📊 Learning Rate: 0.001000
🌟 ¡Nuevo mejor modelo! Val Acc: 63.83%

📍 Época 2/10
----------------------------------------------------------------------


✅ Train Loss: 1.5600 | Train Acc: 50.68%
✅ Val Loss:   1.5243 | Val Acc:   68.09%
📊 Learning Rate: 0.001000
🌟 ¡Nuevo mejor modelo! Val Acc: 68.09%

📍 Época 3/10
----------------------------------------------------------------------


✅ Train Loss: 1.5275 | Train Acc: 61.42%
✅ Val Loss:   1.4857 | Val Acc:   82.98%
📊 Learning Rate: 0.001000
🌟 ¡Nuevo mejor modelo! Val Acc: 82.98%

📍 Época 4/10
----------------------------------------------------------------------


✅ Train Loss: 1.4971 | Train Acc: 68.26%
✅ Val Loss:   1.4507 | Val Acc:   87.23%
📊 Learning Rate: 0.001000
🌟 ¡Nuevo mejor modelo! Val Acc: 87.23%

📍 Época 5/10
----------------------------------------------------------------------


...

✅ Train Loss: 1.4514 | Train Acc: 77.40%
✅ Val Loss:   1.4163 | Val Acc:   89.36%
📊 Learning Rate: 0.000100

📍 Época 7/10
----------------------------------------------------------------------


✅ Train Loss: 1.4515 | Train Acc: 76.26%
✅ Val Loss:   1.4154 | Val Acc:   89.36%
📊 Learning Rate: 0.000100

📍 Época 8/10
----------------------------------------------------------------------


✅ Train Loss: 1.4440 | Train Acc: 79.91%
✅ Val Loss:   1.4113 | Val Acc:   87.23%
📊 Learning Rate: 0.000100

📍 Época 9/10
----------------------------------------------------------------------


✅ Train Loss: 1.4460 | Train Acc: 75.80%
✅ Val Loss:   1.4089 | Val Acc:   89.36%
📊 Learning Rate: 0.000100

📍 Época 10/10
----------------------------------------------------------------------


✅ Train Loss: 1.4396 | Train Acc: 79.68%
✅ Val Loss:   1.4044 | Val Acc:   89.36%
📊 Learning Rate: 0.000010

✅ ENTRENAMIENTO COMPLETADO
🏆 Mejor Val Accuracy: 89.36%
✅ Mejor modelo cargado para evaluación


## 📊 Paso 11: Visualizar Curvas de Entrenamiento

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de Loss
axes[0].plot(train_losses, label='Train Loss', marker='o')
axes[0].plot(val_losses, label='Val Loss', marker='s')
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Loss')
axes[0].set_title('Pérdida durante el Entrenamiento')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico de Accuracy
axes[1].plot(train_accs, label='Train Accuracy', marker='o')
axes[1].plot(val_accs, label='Val Accuracy', marker='s')
axes[1].set_xlabel('Época')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Precisión durante el Entrenamiento')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Curvas de entrenamiento visualizadas!")

## 🧪 Paso 12: Evaluación en el Conjunto de Test

In [ ]:
print("="*70)
print("🧪 EVALUACIÓN EN TEST SET")
print("="*70)

# Evaluar en test
test_loss, test_acc = validate(model, test_loader, criterion, device)

print(f"\n✅ Test Loss: {test_loss:.4f}")
print(f"✅ Test Accuracy: {test_acc:.2f}%")
print("="*70)

## 📈 Paso 13: Matriz de Confusión

Visualizamos el desempeño del modelo en cada clase.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Obtener predicciones en test set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

# Crear matriz de confusión
cm = confusion_matrix(all_labels, all_preds)

# Visualizar matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, 
            yticklabels=class_names)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Test Set')
plt.tight_layout()
plt.show()

# Reporte de clasificación
print("\n" + "="*70)
print("📊 REPORTE DE CLASIFICACIÓN")
print("="*70)
print(classification_report(all_labels, all_preds, 
                          target_names=class_names, 
                          digits=4))
print("="*70)

## 💾 Paso 14: Guardar el Modelo

Guardamos el modelo entrenado para uso futuro.

In [ ]:
# Guardar el modelo completo
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'train_accs': train_accs,
    'val_losses': val_losses,
    'val_accs': val_accs,
    'test_acc': test_acc,
    'test_loss': test_loss,
    'class_names': class_names,
    'num_epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE
}, 'shufflenet_modelo_final.pth')

print("✅ Modelo guardado como 'shufflenet_modelo_final.pth'")

## 🎉 Paso 15: Resumen Final del Proyecto

Mostramos un resumen completo de los resultados obtenidos.

In [ ]:
print("="*70)
print("🎉 RESUMEN DEL PROYECTO SHUFFLENET - TRANSFER LEARNING")
print("="*70)

print("\n🔵 CONFIGURACIÓN:")
print(f"   • Modelo base: ShuffleNet V2 x1.0 (ImageNet)")
print(f"   • Clases: {NUM_CLASSES} vegetales")
print(f"   • Batch size: {BATCH_SIZE}")
print(f"   • Learning rate inicial: {LEARNING_RATE}")
print(f"   • Épocas: {NUM_EPOCHS}")
print(f"   • Dispositivo: {device}")

print("\n🔵 DATASET:")
print(f"   • Clases: {', '.join(class_names)}")
print(f"   • Train: {len(train_loader.dataset)} imágenes")
print(f"   • Validation: {len(val_loader.dataset)} imágenes")
print(f"   • Test: {len(test_loader.dataset)} imágenes")

print("\n🔵 ARQUITECTURA:")
params = model.count_parameters()
print(f"   • Parámetros totales: {params['total']:,}")
print(f"   • Parámetros entrenables: {params['trainable']:,} ({params['trainable']/params['total']*100:.2f}%)")
print(f"   • Feature extractor: Congelado")
print(f"   • Clasificador: 1 capa FC (simple)")

print("\n🔵 RESULTADOS FINALES:")
print(f"   • Mejor Val Accuracy: {best_val_acc:.2f}%")
print(f"   • Test Accuracy: {test_acc:.2f}%")
print(f"   • Test Loss: {test_loss:.4f}")

print("\n🔵 ARCHIVOS GENERADOS:")
print(f"   • Modelo guardado: shufflenet_modelo_final.pth")

print("\n" + "="*70)
print("✅ PROYECTO COMPLETADO EXITOSAMENTE")
print("="*70)

print("\n💡 PRÓXIMOS PASOS:")
print("   1. Probar con diferentes hiperparámetros")
print("   2. Implementar clasificador más complejo (BatchNorm, Dropout)")
print("   3. Hacer fine-tuning del feature extractor")
print("   4. Probar con más épocas de entrenamiento")
print("   5. Experimentar con diferentes estrategias de data augmentation")